# Symmetric Eigenvalue Decomposition - Jacobi Method and High Relative Accuracy


The Jacobi method is the oldest method for EVD computations, dating back from 1864. 
The method does not require tridiagonalization.
Instead, the method computes a sequence of orthogonally similar 
matrices which converge to a diagonal matrix of eigenvalues. In each step a simple plane rotation
which sets one off-diagonal element to zero is performed. 

For positive definite matrices, the method computes eigenvalues with high relative accuracy.

For more details, see 
[I. Slapničar, Symmetric Matrix Eigenvalue Techniques][Hog14] and
[Z. Drmač, Computing Eigenvalues and Singular Values to High Relative Accuracy][Hog14a]
and the references therein.

[Hog14]: #1 "L. Hogben, ed., 'Handbook of Linear Algebra', pp. 55.1-55.25, CRC Press, Boca Raton, 2014."

[Hog14a]: #1 "L. Hogben, ed., 'Handbook of Linear Algebra', pp. 59.1-59.21, CRC Press, Boca Raton, 2014."


## Prerequisites

The reader should be familiar with concepts of eigenvalues and eigenvectors, related perturbation theory, and algorithms. 

 
## Competences 

The reader should be able to recognise matrices which warrant high relative accuracy and to apply 
Jacobi method to them.


## Jacobi method

$A$ is a real symmetric matrix of order $n$ and $A= U \Lambda  U^T$ is its EVD.

### Definitions

The __Jacobi method__ forms a sequence of matrices,

$$
A_0=A, \qquad A_{k+1}=G(c,s,i_k,j_k) A_k G(c,s,i_k,j_k)^T, \qquad
k=1,2,\ldots,
$$

where $G(c,s,i_k,j_k)$ is the orthogonal __plane rotation matrix__.
The parameters $c$ and $s$ are chosen such that 

$$
[A_{k+1}]_{i_k j_k}=[A_{k+1}]_{j_k i_k}=0.
$$

The plane rotation is also called __Jacobi rotation__. 

The __off-norm__ of $A$ is 

$$
\| A\|_{\mathrm{off}}=\big(\sum_{i}\sum_{j\neq i} a_{ij}^2\big)^{1/2},
$$

that is, off-norm is the Frobenius norm of the
matrix consisting of all off-diagonal elements of $A$.

The choice of __pivot elements__ $[A_k]_{i_kj_k}$ is called the 
__pivoting strategy__.

The __optimal pivoting strategy__, originally used by Jacobi, chooses pivoting
elements such that 

$$
|[A_k]_{i_k j_k}|=\max_{i<j} |[A_k]_{ij}|.
$$

The __row-cyclic__ pivoting strategy chooses pivot elements
  in the systematic row-wise order,

$$
(1,2), (1,3), \ldots,(1,n),(2,3),
(2,4),\ldots,(2,n),(3,4),\ldots,(n-1,n).
$$

Similarly, the column-cyclic strategy chooses pivot elements column-wise.

One pass through all matrix elements is called __cycle__ or __sweep__.

### Facts

1. The Jacobi rotations parameters $c$ and $s$ are computed as follows:
 If $[A_k]_{i_kj_k}=0$, then $c=1$ and $s=0$, otherwise
\begin{align*}
& \tau=\frac{[A_k]_{i_ki_k}-[A_k]_{j_kj_k} }{2[A_k]_{i_kj_k} },\qquad
t=\frac{\mathop{\mathrm{sign}}(\tau)}{|\tau|+\sqrt{1+\tau^2}},\\
& c=\frac{1}{\sqrt{1+t^2}},\qquad s=c\cdot t.
\end{align*} 

2. After each rotation, the off-norm decreases,
$$
\|A_{k+1}\|_{\mathrm{off}}^2=\|A_{k}\|_{\mathrm{off}}^2-2[A_k]_{i_kj_k}^2.
$$
With the appropriate pivoting strategy, the method converges in the sense that
$$
\|A_{k}\|_{\mathrm{off}}\to 0,\qquad A_k\to\Lambda, \qquad 
\prod_{k=1}^{\infty} G(i_k,j_k,c,s)^T \to U.
$$

3. For the optimal pivoting strategy
the square of the pivot element is greater than the average squared
element, 
$$
[A_k]_{i_kj_k}^2\geq \frac{1}{n(n-1)}\,
\|A_k\|_{\mathrm{off}}^2 .
$$
Thus,
$$
\|A_{k+1}\|_{\mathrm{off}}^2\leq\left(1-\frac{2}{n(n-1)}\right)\|A_{k}\|_{\mathrm{off}}^2
$$
and the method converges.

4. For the row cyclic and the column cyclic pivoting strategies, the method
converges. The convergence is ultimately __quadratic__ in the sense that
$$
\|A_{k+n(n-1)/2}\|_{\mathrm{off}} \leq\ const\cdot  \|A_{k}\|_{\mathrm{off}}^2,
$$
provided $\|A_{k}\|_{\mathrm{off}}$ is sufficiently small.

5. The EVD computed by the Jacobi method satisfies the standard error bounds.

6. The Jacobi method is suitable for parallel computation. There exist convergent parallel
strategies which enable simultaneous execution of several rotations.
  
7. The Jacobi method is simple, but it is slower than the methods based on tridiagonalization. It is
conjectured that standard implementations require $O(n^3\log n)$ operations. More precisely, each cycle clearly requires $O(n^3)$ operations and it is conjectured that $\log n$ cycles are needed until convergence.
 
8. If $A$ is positive definite, the method can be modified such that it reaches
the speed of the methods based on tridiagonalization and at the same time
computes the EVD with high relative accuracy.

### Examples

In [1]:
function myJacobi{T}(A::Array{T})
    n=size(A,1)
    U=eye(T,n)
    # Tolerance for rotation
    tol=sqrt(n)*eps(T)
    # Counters
    p=n*(n-1)/2
    sweep=0
    pcurrent=0
    # First criterion is for standard accuracy, second one is for relative accuracy
    # while sweep<30 && vecnorm(A-diagm(diag(A)))>tol
    while sweep<30 && pcurrent<p
        sweep+=1
        # Row-cyclic strategy
        for i = 1 : n-1 
            for j = i+1 : n
                # Check for the tolerance - the first criterion is standard,
                # the second one is for relative accuracy for PD matrices               
                # if A[i,j]!=zero(T)
                if abs(A[i,j])>tol*sqrt(abs(A[i,i]*A[j,j]))
                    # Compute c and s
                    τ=(A[i,i]-A[j,j])/(2*A[i,j])
                    t=sign(τ)/(abs(τ)+sqrt(1+τ^2))
                    c=1/sqrt(1+t^2)
                    s=c*t
                    G=LinAlg.Givens(i,j,c,s)
                    A=G*A
                    A*=G'
                    A[i,j]=zero(T)
                    A[j,i]=zero(T)
                    U*=G'
                    pcurrent=0
                else
                    pcurrent+=1
                end
            end
        end
    end
    diag(A), U
end

myJacobi (generic function with 1 method)

In [2]:
n=4
s=srand(421)
A=full(Symmetric(rand(n,n)))

4×4 Array{Float64,2}:
 0.345443  0.105135  0.785847  0.608612
 0.105135  0.77247   0.135538  0.346561
 0.785847  0.135538  0.958365  0.561248
 0.608612  0.346561  0.561248  0.915812

In [3]:
λ,U=myJacobi(A)

([-0.239024, 0.76775, 2.15297, 0.310394], [0.855237 -0.20096 0.476545 -0.0329968; 0.0566885 0.881773 0.242367 -0.40065; -0.451271 -0.378529 0.613859 -0.525592; -0.248415 0.196996 0.580809 0.749762])

In [4]:
# Orthogonality
U'*U

4×4 Array{Float64,2}:
  1.0           7.00964e-17  -4.25496e-17  -5.21254e-17
  7.00964e-17   1.0          -2.07554e-17   5.2896e-17 
 -4.25496e-17  -2.07554e-17   1.0           4.2885e-17 
 -5.21254e-17   5.2896e-17    4.2885e-17    1.0        

In [5]:
# Residual
A*U-U*diagm(λ)

4×4 Array{Float64,2}:
  2.77556e-17  -2.498e-16    -1.11022e-15  -1.21431e-17
  3.29597e-17   5.55112e-16  -7.77156e-16  -4.16334e-17
  6.93889e-17  -2.77556e-16  -1.55431e-15   0.0        
 -1.17961e-16   5.55112e-17  -1.33227e-15   3.05311e-16

In [6]:
# Positive definite matrix
n=100
A=rand(n,n)
A=full(Symmetric(A'*A));

In [7]:
λ,U=myJacobi(A)
vecnorm(U'*U-I),vecnorm(A*U-U*diagm(λ))

(1.7388038681132419e-13, 2.806431003268346e-11)

In [8]:
λ

100-element Array{Float64,1}:
 2501.5       
    0.166124  
   31.7579    
    7.50007e-6
   19.7288    
    1.96576   
    0.0600245 
    0.0834034 
   28.8219    
    0.00403152
   30.5573    
   27.315     
   19.1515    
    ⋮         
    5.44234   
    2.24968   
    2.86735   
    2.69984   
    3.2681    
    4.89029   
    3.72714   
    4.39552   
    4.6202    
    3.12447   
    3.98454   
    4.7994    

In [9]:
cond(A)

3.3353082719919485e8

In [10]:
# Now the standard QR method
λₛ,Uₛ=eig(A);

In [11]:
vecnorm(Uₛ'*Uₛ-I),vecnorm(A*Uₛ-Uₛ*diagm(λₛ))

(1.0975933228075441e-13, 2.116710049248954e-12)

`myJacobi()` is accurate but very slow. Notice the extreemely high memory allocation.

The two key elements to reducing the allocations are: 
1. make sure variables don't change type within a function, and  
2. reuse arrays in hot loops.

Here we will simply use the in-place multiplication routines which are in Julia denoted by `!`.

In [12]:
@time eig(A);

  0.005076 seconds (106 allocations: 278.623 KiB)


In [13]:
@time myJacobi(A);

  4.158694 seconds (224.08 k allocations: 8.356 GiB, 22.10% gc time)


In [14]:
methodswith(LinAlg.Givens)

10-element Array{Method,1}:
 *(G1::Base.LinAlg.Givens{T}, G2::Base.LinAlg.Givens{T}) where T in Base.LinAlg at linalg\givens.jl:363                          
 A_mul_B!(G1::Base.LinAlg.Givens, G2::Base.LinAlg.Givens) in Base.LinAlg at linalg\givens.jl:321                                 
 A_mul_B!(G::Base.LinAlg.Givens, A::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T) in Base.LinAlg at linalg\givens.jl:324
 A_mul_B!(G::Base.LinAlg.Givens, R::Base.LinAlg.Rotation) in Base.LinAlg at linalg\givens.jl:348                                 
 A_mul_Bc!(A::AbstractArray{T,2} where T, G::Base.LinAlg.Givens) in Base.LinAlg at linalg\givens.jl:336                          
 convert(::Type{Base.LinAlg.Givens{T}}, G::Base.LinAlg.Givens{T}) where T in Base.LinAlg at linalg\givens.jl:37                  
 convert(::Type{Base.LinAlg.Givens{T}}, G::Base.LinAlg.Givens) where T in Base.LinAlg at linalg\givens.jl:38                     
 convert(::Type{Base.LinAlg.AbstractRotation{T}}, G::Base.LinAlg.Givens) where T in Base.LinAlg at linalg\givens.jl:41           
 ctranspose(G::Base.LinAlg.Givens) in Base.LinAlg at linalg\givens.jl:44                                                         
 getindex(G::Base.LinAlg.Givens, i::Integer, j::Integer) in Base.LinAlg at linalg\givens.jl:305

In [15]:
function myJacobi{T}(A::Array{T})
    n=size(A,1)
    U=eye(T,n)
    # Tolerance for rotation
    tol=sqrt(n)*eps(T)
    # Counters
    p=n*(n-1)/2
    sweep=0
    pcurrent=0
    # First criterion is for standard accuracy, second one is for relative accuracy
    # while sweep<30 && vecnorm(A-diagm(diag(A)))>tol
    while sweep<30 && pcurrent<p
        sweep+=1
        # Row-cyclic strategy
        for i = 1 : n-1 
            for j = i+1 : n
                # Check for the tolerance - the first criterion is standard,
                # the second one is for relative accuracy for PD matrices               
                # if A[i,j]!=zero(T)
                if abs(A[i,j])>tol*sqrt(abs(A[i,i]*A[j,j]))
                    # Compute c and s
                    τ=(A[i,i]-A[j,j])/(2*A[i,j])
                    t=sign(τ)/(abs(τ)+sqrt(1+τ^2))
                    c=1/sqrt(1+t^2)
                    s=c*t
                    G=LinAlg.Givens(i,j,c,s)
                    # A=G*A
                    A_mul_B!(G,A)
                    # A*=G'
                    A_mul_Bc!(A,G)
                    A[i,j]=zero(T)
                    A[j,i]=zero(T)
                    # U*=G'
                    A_mul_Bc!(U,G)
                    pcurrent=0
                else
                    pcurrent+=1
                end
            end
        end
    end
    diag(A), U
end

myJacobi (generic function with 1 method)

In [17]:
@time λ,U=myJacobi(A);

  0.000093 seconds (12 allocations: 79.391 KiB)


In [18]:
vecnorm(U'*U-I),vecnorm(A*U-U*diagm(λ))

(0.0, 5.923810304938232e-13)

## Relative perturbation theory

$A$  is a real symmetric PD matrix of order $n$  and $A=U\Lambda U^T$ is its EVD.

### Definition

The __scaled matrix__ of the matrix $A$ is the matrix
$$
A_S=D^{-1} A D^{-1}, \quad D=\mathop{\mathrm{diag}}(\sqrt{A_{11}},\sqrt{A_{22}},\ldots,\sqrt{A_{nn}}).
$$

### Facts

1. The above diagonal scaling is nearly optimal (van der Sluis):
$$
\kappa_2(A_S)\leq  n \min\limits_{D=\mathrm{diag}} \kappa(DHD) \leq n\kappa_2(H).
$$

2. Let $A$ and $\tilde A=A+\Delta A$ both be positive definite, and let 
their eigenvalues have the same ordering. Then
$$
\frac{|\lambda_i-\tilde\lambda_i|}{\lambda_i}\leq 
\frac{\| D^{-1} (\Delta A) D^{-1}\|_2}{\lambda_{\min} (A_S)}\equiv
\|A_S^{-1}\|_2 \| \Delta A_S\|_2.
$$
If $\lambda_i$ and $\tilde\lambda_i$ are simple, then
$$
\|U_{:,i}-\tilde U_{:,i}\|_2 \leq \frac{\| A_S^{-1}\|_2 \|\Delta A_S\|_2}
{\displaystyle\min_{j\neq i}\frac{|\lambda_i-\lambda_j|}{\sqrt{\lambda_i\lambda_j}}}.
$$
These bounds are much sharper than the standard bounds for matrices for which $\kappa_2(A_S)\ll \kappa_2(A)$.

3. The Jacobi method with the relative stopping criterion 
$$
|A_{ij}|\leq tol \sqrt{A_{ii}A_{jj}}, \quad \forall $i\neq j,
$$
and some user defined tolerance $tol$ (usually $tol=n\varepsilon$), computes the EVD with small scaled  backward error
$$
\|\Delta A_S\|\leq \varepsilon\, O(\|A_S\|_2)\leq O(n)\varepsilon,
$$
_provided_ that $\kappa_2([A_k]_S)$  does not grow much during the iterations. There is overwhelming numerical evidence that the scaled condition does not grow much, and the growth can be monitored, as well.

The proofs of the above facts are in [J. Demmel and K. Veselić, Jacobi's method is more accurate than QR](http://www.netlib.org/lapack/lawnspdf/lawn15.pdf).  

### Example - Scaled matrix


In [19]:
n=6
A=rand(n,n)
A=full(Symmetric(A'*A));
Aₛ=[A[i,j]/sqrt(A[i,i]*A[j,j]) for i=1:n, j=1:n]

6×6 Array{Float64,2}:
 1.0       0.832247  0.975987  0.70636   0.736069  0.460841
 0.832247  1.0       0.908833  0.70454   0.651929  0.715246
 0.975987  0.908833  1.0       0.731738  0.704186  0.519369
 0.70636   0.70454   0.731738  1.0       0.939989  0.574457
 0.736069  0.651929  0.704186  0.939989  1.0       0.472141
 0.460841  0.715246  0.519369  0.574457  0.472141  1.0     

In [20]:
cond(Aₛ), cond(A)

(4961.858340517723, 5172.032232802231)

In [28]:
# We add a strong scaling
D=exp.(50*(rand(n)-0.5))

6-element Array{Float64,1}:
 0.000288221
 1.11513e9  
 1.68327e7  
 3.07436e7  
 2.48905e8  
 0.000416369

In [29]:
H=diagm(D)*Aₛ*diagm(D)

6×6 Array{Float64,2}:
     8.30715e-8  2.67488e5   4735.03        …  52805.3             5.53038e-8
     2.67488e5   1.24352e18     1.70594e16         1.8095e17       3.32093e5 
  4735.03        1.70594e16     2.83339e14         2.95035e15   3640.05      
  6259.01        2.41539e16     3.78671e14         7.19299e15   7353.43      
 52805.3         1.8095e17      2.95035e15         6.19535e16  48930.8       
     5.53038e-8  3.32093e5   3640.05        …  48930.8             1.73363e-7

In [30]:
# Now we scale again
Hₛ=[H[i,j]/sqrt(H[i,i]*H[j,j]) for i=1:n, j=1:n]

6×6 Array{Float64,2}:
 1.0       0.832247  0.975987  0.70636   0.736069  0.460841
 0.832247  1.0       0.908833  0.70454   0.651929  0.715246
 0.975987  0.908833  1.0       0.731738  0.704186  0.519369
 0.70636   0.70454   0.731738  1.0       0.939989  0.574457
 0.736069  0.651929  0.704186  0.939989  1.0       0.472141
 0.460841  0.715246  0.519369  0.574457  0.472141  1.0     

In [31]:
cond(Hₛ),cond(H)

(4961.858340517125, 4.731573875018593e27)

In [32]:
# Jacobi method
λ,U=myJacobi(H)

([2.58678e-10, 1.27134e18, 4.3125e13, 9.521e13, 3.52272e16, 5.8448e-8], [0.995468 2.14317e-13 … 3.58808e-13 0.0950961; 1.11316e-13 0.988687 … -0.149468 -4.69049e-13; … ; -5.50367e-13 0.148079 … 0.983513 5.33501e-13; -0.0950961 2.64112e-13 … -2.06829e-14 0.995468])

In [33]:
# Standard QR method
λ₁,U₁=eig(H)

([2.58678e-10, 3.52272e16, 1.27134e18, -27.2987, 9.521e13, 4.3125e13], [1.0 1.19226e-33 … -1.05236e-51 5.52641e-51; 0.0 -2.25514e-17 … 1.07704e-17 7.69375e-17; … ; 0.0 1.0 … -4.8831e-22 2.33579e-23; 0.0 1.15899e-20 … -4.27678e-14 1.27373e-13])

In [34]:
# Compare
[sort(λ) sort(λ₁)]

6×2 Array{Float64,2}:
 2.58678e-10  -27.2987     
 5.8448e-8      2.58678e-10
 4.3125e13      4.3125e13  
 9.521e13       9.521e13   
 3.52272e16     3.52272e16 
 1.27134e18     1.27134e18 

In [35]:
# Check with BigFloat
λ₂,U₂=myJacobi(map(BigFloat,H))
λ₂

6-element Array{BigFloat,1}:
 2.586775805334472301991069895573760220214899900298389688281267417091959819130086e-10
 1.271336908422575360000000000000000000000000123687681416667162496048825461925205e+18
 4.312498789789791406249999999999999999987631231858333291950247578990208235978315e+13
 9.521002910696420312499999999999999999999158656797216545143546486490978155251381e+13
 3.522721780123314000000000000000000000000000841343202784095619769482696255295215e+16
 5.84480115910659981301682305756336570645352882445640785875330471887816464011359e-08 

In [36]:
# Relative error is eps()*cond(AS)
map(Float64,(sort(λ₂)-sort(λ))./sort(λ₂))

6-element Array{Float64,1}:
 -2.3065e-31 
 -1.10012e-31
 -2.86812e-39
 -8.83671e-41
  2.38833e-43
  9.72895e-44

## Indefinite matrices

### Definition

__Spectral absolute value__ of the matrix $A$ is the matrix 

$$
|A|_{\mathrm{spr}}=(A^2)^{1/2}.
$$

This is positive definite part of the polar decomposition of $A$.

### Facts

1. The above perturbation bounds for positive definite matrices essentially hold with $A_S$ replaced by $[|A|_{\mathrm{spr}}]_S$.

2. Jacobi method can be modified to compute the EVD with small backward error 
$\| \Delta [|A|_{\mathrm{spr}}]_S\|_2$.

The details of the indefinite case are beyond the scope of this course, and the reader should consider references.